<a href="https://colab.research.google.com/github/DayalStrub/ecir2021tutorial/blob/main/other/2-simplest-thing-WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q python-terrier
!pip install -q git+https://github.com/terrier-org/pyterrier.git

     |████████████████████████████████| 1.1MB 18.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 163kB 45.2MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 1.8MB 45.8MB/s 
     |████████████████████████████████| 5.5MB 45.5MB/s 
     |████████████████████████████████| 133kB 36.5MB/s 
     |████████████████████████████████| 645kB 41.8MB/s 


In [12]:
import pandas as pd

In [2]:
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

In [4]:
next(iter(dataset.get_corpus_iter()))

[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
[INFO] [finished] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: [00:09] [269MB] [29.2MB/s]
[INFO] [starting] building docstore
docs_iter: 192509it [00:12, 15079.90it/s]
[INFO] [finished] docs_iter: [00:12] [192509it] [15072.20it/s]
[INFO] [finished] building docstore [12.78s]


OrderedDict([('title',
              'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia'),
             ('doi', '10.1186/1471-2334-1-6'),
             ('date', '2001-07-04'),
             ('abstract',
              'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common

In [5]:
N = 10_000
pt_index_path = './index_cord19'

# create the index, using the IterDictIndexer indexer 
indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True) # , blocks=True

# we give the dataset get_corpus_iter() directly to the indexer
# while specifying the fields to index and the metadata to record
index_ref = indexer.index(
    (datum for i, datum in enumerate(dataset.get_corpus_iter()) if i < N),
    fields=('abstract',), # TODO should this be a list?
    # meta={'docno' : 26, 'text' : 2048, 'abstract' : 2048}
    meta=['docno','abstract'],
    )

index = pt.IndexFactory.of(index_ref)

06:27:40.646 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed

06:27:53.243 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 2143 empty documents


In [ ]:
print(index.getCollectionStatistics().toString())

Number of documents: 10000
Number of terms: 36191
Number of postings: 673758
Number of fields: 1
Number of tokens: 1040094
Field names: [abstract]
Positions:   false



In [6]:
queries = dataset.get_topics(variant='title').head(1)
queries.head()

[INFO] If you have a local copy of https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/0307a37b6b9f1a5f233340a769d538ea
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: [00:00] [18.7kB] [5.45MB/s]


,qid,query
0,1,coronavirus origin


In [82]:
M = 1001 # TODO is 1000 the max ever returned

bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=M)
rm3 = pt.rewrite.RM3(index)
bm25_text = pt.BatchRetrieve(index, wmodel="BM25", metadata=["docno", "abstract"], num_results=M)

rm3_pipe = bm25 >> rm3 >> bm25_text

In [83]:
# TODO Question: in rm3_pipe, is final bm25 just reranking initial bm25 results, or does it re-consider all docs?

In [84]:
# TODO figure out how to implement PR Feedback using logistic regression on bag of words a la Lin

In [85]:
df_out = rm3_pipe.transform(queries)
df_out.loc[df_out["qid"] == "1", :]

,qid,docid,docno,abstract,rank,score,query_0,query
0,1,2257,8y24j34j,Middle East respiratory syndrome coronavirus i...,0,19.049835,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
1,1,4582,2jq626ye,A novel coronavirus (2019-nCoV) originating in...,1,13.717366,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
2,1,1394,vnafx1ng,The phosphoprotein (P) gene of most Paramyxovi...,2,11.114468,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
3,1,1616,hp5x637c,"BACKGROUND: In 2007, a novel bunyavirus was fo...",3,8.417806,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
4,1,4936,41ui4lqc,Emerging viruses represent a continuous threat...,4,8.340423,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
...,...,...,...,...,...,...,...,...
995,1,1727,5d64esp4,"RNA viruses exhibit substantial structural, ec...",995,0.986444,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
996,1,1685,bkreu1s2,"Collectively, viruses have the greatest geneti...",996,0.986241,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
997,1,7870,o2cijp5a,In the present report we provide the strain di...,997,0.986241,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
998,1,4489,78ur1j2p,Despite all the research done on the first Zik...,998,0.983408,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...


In [65]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [66]:
n_top = 10
n_bottom = 100 # TODO should actually get 0 score documents? how to do this in pyterrier?

col_text = 'abstract'

df_relevant = df_out.sort_values("rank").loc[0:n_top, ["docid", "abstract"]]
df_relevant["label"] = 1

df_not_relevant = df_out.sort_values("rank", ascending=False).reset_index(drop=True).loc[0:n_bottom, ["docid", "abstract"]]
df_not_relevant["label"] = 0

df_train = pd.concat([df_relevant, df_not_relevant])

text_transformer = Pipeline([
    ('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
])

preprocessor = ColumnTransformer(
    transformers=[
        # ('num', num_transformer, col_num),
        # ('cat', cat_transformer, col_cat),
        ('text', text_transformer, col_text)
    ]
)

crf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

X_train = df_train.loc[:, ["docid", "abstract"]]
y_train = df_train["label"]

crf.fit(X_train, y_train) #;

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('text',
                                                  Pipeline(memory=None,
                                                           steps=[('bow',
                                                                   CountVectorizer(analyzer='word',
                                                                                   binary=False,
                                                                                   decode_error='strict',
                                                                                   dtype=<class 'numpy.int64'>,
                                                                                   encoding='utf-8',
                                         

In [67]:
preds = crf.predict_proba(df_sort.loc[:, ["docid", "abstract"]])

df_out["score"] = preds[:, 1] # True class first?
df_out.sort_values("score", ascending=False, inplace=True)
df_out.reset_index(drop=True, inplace=True)
df_out["rank"] = df_out.index
df_out

,qid,docid,docno,abstract,rank,score,query_0,query
0,1,2257,8y24j34j,Middle East respiratory syndrome coronavirus i...,0,0.208616,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
1,1,1616,hp5x637c,"BACKGROUND: In 2007, a novel bunyavirus was fo...",1,0.206782,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
2,1,5200,6io5yxgo,Many emerging infectious diseases in human pop...,2,0.204686,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
3,1,3012,6eb2jgl7,"Norovirus infection, a highly prevalent condit...",3,0.204143,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
4,1,1574,ey5xmb8a,"Passive immunity (PI), acquired through colost...",4,0.203095,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
...,...,...,...,...,...,...,...,...
995,1,747,ocp6yodg,BACKGROUND: During the initial containment pha...,995,0.080233,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
996,1,5841,pzhi491i,The remarkable transition of biological scienc...,996,0.079512,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
997,1,4904,qote5nx2,The genetically selected high antibody respond...,997,0.079462,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
998,1,156,iy4c7404,"In networks, nodes may preferentially contact ...",998,0.079132,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...


In [68]:
def pr_linear_reranker(
    df,
    *,
    n_top = 10,
    n_bottom = 100, # TODO should actually get 0 score documents? how to do this in pyterrier?
    col_text = 'abstract'
):
  df_tmp = df.copy()
  df_relevant = df_tmp.sort_values("rank").loc[0:n_top, ["docid", "abstract"]]
  df_relevant["label"] = 1

  df_not_relevant = df_tmp.sort_values("rank", ascending=False).reset_index(drop=True).loc[0:n_bottom, ["docid", "abstract"]]
  df_not_relevant["label"] = 0

  df_train = pd.concat([df_relevant, df_not_relevant])

  text_transformer = Pipeline([
      ('bow', CountVectorizer()),
      ('tfidf', TfidfTransformer()),
  ])

  preprocessor = ColumnTransformer(
      transformers=[
          # ('num', num_transformer, col_num),
          # ('cat', cat_transformer, col_cat),
          ('text', text_transformer, col_text)
      ]
  )

  crf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression())
  ])

  X_train = df_train.loc[:, ["docid", "abstract"]]
  y_train = df_train["label"]

  crf.fit(X_train, y_train);

  preds = crf.predict_proba(df_tmp.loc[:, ["docid", "abstract"]])

  df_tmp["score"] = preds[:, 1] # True class first?
  df_tmp.sort_values("score", ascending=False, inplace=True)
  df_tmp.reset_index(drop=True, inplace=True)
  df_tmp["rank"] = df_out.index
  return df_tmp

In [69]:
pr_linear_reranker(df_out)

,qid,docid,docno,abstract,rank,score,query_0,query
0,1,2257,8y24j34j,Middle East respiratory syndrome coronavirus i...,0,0.248143,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
1,1,1616,hp5x637c,"BACKGROUND: In 2007, a novel bunyavirus was fo...",1,0.237354,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
2,1,1574,ey5xmb8a,"Passive immunity (PI), acquired through colost...",2,0.236559,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
3,1,5200,6io5yxgo,Many emerging infectious diseases in human pop...,3,0.235666,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
4,1,3012,6eb2jgl7,"Norovirus infection, a highly prevalent condit...",4,0.233275,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
...,...,...,...,...,...,...,...,...
995,1,4904,qote5nx2,The genetically selected high antibody respond...,995,0.079123,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
996,1,755,0qkzd2w4,Angiotensin (Ang)-(17) is now recognized as a ...,996,0.078087,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
997,1,1524,mx4t6s6n,Hypertension is a major risk factor for stroke...,997,0.076945,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...
998,1,2657,b0p1b4g8,An exacerbated immune response is one of the m...,998,0.076342,coronavirus origin,applypipeline:off protect^0.035425290 anim^0.0...


In [ ]:
# TODO scale RM3 and PR_linear

In [ ]:
pt.apply.by_query()

In [ ]:
# NOTE indeed/actually a function
pt.Experiment(
    retr_systems=[bm25, rm3_pipe],
    names=["BM25", "BM25-RM3-BM25",],
    topics=queries,
    qrels=dataset.get_qrels(), # TODO figure out cord19 qrels
    eval_metrics=["map"]
              )

,name,map
0,BM25,0.000020
1,BM25-RM3-BM25,0.000018
